In [1]:
# legislator data source: https://github.com/unitedstates/congress-legislators

In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json

### Read in Biographical Data from json source files

In [2]:
def convert_json_to_df(filename):
    with open(filename, "r") as file:
        data = json.load(file)
        
    df = pd.json_normalize(data, max_level=1)
    df['terms'] = df['terms'].apply(lambda x: x[0])
    terms_df = pd.json_normalize(df['terms'], max_level=1)
    try:
        terms_df = terms_df[['type', 'party', 'start', 'end', 'state']]
    except: # presidents don't have 'state' data
        terms_df = terms_df[['type', 'party', 'start', 'end']]
    legislator_df = pd.concat([df, terms_df], axis=1)
    try:
        legislator_df = legislator_df[['id.bioguide', 'id.wikipedia', 'name.first', 'name.middle','name.last', 
                                   'bio.gender', 'bio.birthday', 'type', 'party', 'start', 'end', 'state']]
    except: # presidents don't have 'state' data
        legislator_df = legislator_df[['id.bioguide', 'id.wikipedia', 'name.first', 'name.middle','name.last', 
                                   'bio.gender', 'bio.birthday', 'type', 'party', 'start', 'end']]
    return legislator_df

In [3]:
# historical_legislator_df = pd.read_json("https://theunitedstates.io/congress-legislators/legislators-historical.json")
historical_legislator_df = convert_json_to_df("legislators-historical.json")
# current_legislator_df = pd.read_json("https://theunitedstates.io/congress-legislators/legislators-current.json")
current_legislator_df = convert_json_to_df("legislators-current.json")
# presidents_df = pd.read_json("https://theunitedstates.io/congress-legislators/executive.json")
presidents_df = convert_json_to_df("presidents-and-vice-presidents.json")

In [4]:
print(f'historical_legislator_df: {historical_legislator_df.shape}')
print(f'current_legislator_df: {current_legislator_df.shape}')
print(f'presidents_df: {presidents_df.shape}')

historical_legislator_df: (12058, 12)
current_legislator_df: (539, 12)
presidents_df: (79, 11)


### Combine All Biographical Data for the different legislators

In [5]:
all_legislators_df = pd.concat([historical_legislator_df, current_legislator_df, presidents_df])

In [6]:
all_legislators_df.isnull().sum()

id.bioguide       13
id.wikipedia      77
name.first         0
name.middle     3987
name.last          0
bio.gender         0
bio.birthday     550
type               0
party            218
start              0
end                0
state             79
dtype: int64

In [7]:
all_legislators_df.tail()

,id.bioguide,id.wikipedia,name.first,name.middle,name.last,bio.gender,bio.birthday,type,party,start,end,state
74,B000444,Joe Biden,Joseph,Robinette,Biden,M,1942-11-20,viceprez,Democrat,2009-01-20,2013-01-20,NaN
75,O000167,NaN,Barack,Hussein,Obama,M,1961-08-04,prez,Democrat,2009-01-20,2013-01-20,NaN
76,P000587,Mike Pence,Mike,NaN,Pence,M,1959-06-07,viceprez,Republican,2017-01-20,2021-01-20,NaN
77,NaN,Donald Trump,Donald,J.,Trump,M,1946-06-14,prez,Republican,2017-01-20,2021-01-20,NaN
78,H001075,Kamala Harris,Kamala,D.,Harris,F,1964-10-20,viceprez,Democrat,2021-01-20,2025-01-20,NaN


### Create new feature `full_name` on which all_legislators_df will be merged with the original PolitiFact dataset

In [8]:
all_legislators_df['full_name'] = all_legislators_df['name.first'] + ' ' + all_legislators_df['name.last']
all_legislators_df

,id.bioguide,id.wikipedia,name.first,name.middle,name.last,bio.gender,bio.birthday,type,party,start,end,state,full_name
0,B000226,Richard Bassett (Delaware politician),Richard,NaN,Bassett,M,1745-04-02,sen,Anti-Administration,1789-03-04,1793-03-03,DE,Richard Bassett
1,B000546,Theodorick Bland (congressman),Theodorick,NaN,Bland,M,1742-03-21,rep,NaN,1789-03-04,1791-03-03,VA,Theodorick Bland
2,B001086,Aedanus Burke,Aedanus,NaN,Burke,M,1743-06-16,rep,NaN,1789-03-04,1791-03-03,SC,Aedanus Burke
3,C000187,Daniel Carroll,Daniel,NaN,Carroll,M,1730-07-22,rep,NaN,1789-03-04,1791-03-03,MD,Daniel Carroll
4,C000538,George Clymer,George,NaN,Clymer,M,1739-03-16,rep,NaN,1789-03-04,1791-03-03,PA,George Clymer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,B000444,Joe Biden,Joseph,Robinette,Biden,M,1942-11-20,viceprez,Democrat,2009-01-20,2013-01-20,NaN,Joseph Biden
75,O000167,NaN,Barack,Hussein,Obama,M,1961-08-04,prez,Democrat,2009-01-20,2013-01-20,NaN,Barack Obama
76,P000587,Mike Pence,Mike,NaN,Pence,M,1959-06-07,viceprez,Republican,2017-01-20,2021-01-20,NaN,Mike Pence
77,NaN,Donald Trump,Donald,J.,Trump,M,1946-06-14,prez,Republican,2017-01-20,2021-01-20,NaN,Donald Trump


## Merge all_legislator_df with original PolitiFact dataset by matching on legislator name

In [9]:
politifact_df = pd.read_csv('politifact_data.csv')
politifact_df.shape # invalid accuracy ratings already dropped

(25923, 7)

In [10]:
politifact_df['accuracy rating'].unique()

array(['barely-true', 'mostly-true', 'FALSE', 'half-true', 'TRUE',
       'pants-fire'], dtype=object)

In [11]:
politifact_df.head()

,Unnamed: 0,name,claim date,claim source,claim,issue,accuracy rating
0,0,Mandela Barnes,"August 17, 2022",Twitter,“Ron Johnson just came out in favor of a feder...,abortion,barely-true
1,1,CatholicVote,"August 17, 2022",an ad,Three churches were targeted during arson atta...,abortion,barely-true
2,2,Mark Kelly,"August 5, 2022",an ad,Blake Masters “wants to pass a national ban on...,abortion,mostly-true
3,3,Stacey Abrams,"August 4, 2022",an ad,"""Brian Kemp wants to investigate and punish wo...",abortion,FALSE
4,4,Beto O'Rourke,"June 27, 2022",an Instagram post,"""Under Abbott, Texas leads the nation in rape ...",abortion,mostly-true


#### left join politifact_df with all_legislators_df

In [12]:
# left join politifact_df with all_legislators_df where name matches 
all_legislators_df = all_legislators_df.drop_duplicates(subset=['full_name'])
left_joined_df = pd.merge(politifact_df, all_legislators_df, left_on='name', right_on='full_name', how='left')

In [13]:
left_joined_df.shape

(25923, 20)

#### inner join politifact_df with all_legislators_df

In [14]:
# inner join politifact_df with all_legislators_df where name matches 
all_legislators_df = all_legislators_df.drop_duplicates(subset=['full_name'])
inner_joined_df = pd.merge(politifact_df, all_legislators_df, left_on='name', right_on='full_name', how='inner')

In [15]:
inner_joined_df.shape

(7645, 20)

## Check dataframes 

### I. Check left_joined_df

In [16]:
# keep running to check diff rows
left_joined_df[['name', 'bio.gender', 'bio.birthday', 'party']].sample(10)

,name,bio.gender,bio.birthday,party
21189,Americans for Prosperity,NaN,NaN,NaN
8020,John Padgett,NaN,NaN,NaN
16665,Americans for Prosperity: New Jersey,NaN,NaN,NaN
18992,Jim Webb,M,1946-02-09,Democrat
4747,Barack Obama,M,1961-08-04,Democrat
19739,Tdtalliance.com,NaN,NaN,NaN
19929,Donald Trump,M,1946-06-14,Republican
25846,David Quiroa,NaN,NaN,NaN
13261,Jesse Ventura,NaN,NaN,NaN
6426,Rick Santorum,NaN,NaN,NaN


In [17]:
left_joined_df.isnull().sum()

Unnamed: 0             0
name                   0
claim date             0
claim source         138
claim                  0
issue                  0
accuracy rating        0
id.bioguide        19460
id.wikipedia       18278
name.first         18278
name.middle        22474
name.last          18278
bio.gender         18278
bio.birthday       18307
type               18278
party              18281
start              18278
end                18278
state              19460
full_name          18278
dtype: int64

In [18]:
# 18307 null values for `bio.birthday`
bday_perc_null = (18307/left_joined_df.shape[0]) * 100
print(f'{round(bday_perc_null,2)}% of left_joined_df do not have values for `bio.birthday`.')

70.62% of left_joined_df do not have values for `bio.birthday`.


In [19]:
# names that were not matched
left_joined_df.loc[left_joined_df['full_name'].isna()]['name'].unique().tolist()

['Mandela Barnes',
 'CatholicVote',
 'Stacey Abrams',
 "Beto O'Rourke",
 'Karine Jean-Pierre',
 'Kat Cammack',
 'Blake Masters',
 'National Republican Congressional Committee',
 'Dave LaRock',
 'Instagram posts',
 'Planned Parenthood Votes',
 'Kathy Hochul',
 'Catherine Glenn Foster',
 'Focus on the Family',
 'Bob Good',
 'Nikki Fried',
 'Facebook posts',
 'Viral image',
 'Jonathan Turley',
 'Joe Biden',
 'Tweets',
 'Tony Evers',
 "Young America's Foundation",
 'Candace Owens',
 'Kelda Helen Roys',
 'Dave McCormick',
 'Peggy Hubbard',
 'Pam Keith',
 'Andrea Stewart-Cousins',
 'National Republican Senatorial Committee',
 'Don Beyer',
 'Jeffrey Toobin',
 'Jeanine Pirro',
 'Tate Reeves',
 'Terry McAuliffe',
 'Greg Abbott',
 'Occupy Democrats',
 'Derrick Van Orden',
 'Dean Arp',
 'The Family Foundation',
 'Bloggers',
 'Republican National Committee',
 'Nikki Haley',
 'GayDonna Vandergriff',
 'Association of Mature American Citizens',
 'Kelly McGinn',
 'Siobhan Dunnavant',
 'Jeanne Ives',
 

### II. Check inner_joined_df

In [20]:
# keep running to check diff rows
inner_joined_df[['name', 'bio.gender', 'bio.birthday', 'party']].sample(10)

,name,bio.gender,bio.birthday,party
5952,Scott Fitzgerald,M,1963-11-16,Republican
4325,Rick Scott,M,1952-12-01,Republican
5685,Patrick Kennedy,M,1967-07-14,Democrat
4330,Rick Scott,M,1952-12-01,Republican
7415,Tom Marino,M,1952-08-15,Republican
766,John Cornyn,M,1952-02-02,Republican
2032,Kamala Harris,F,1964-10-20,Democrat
769,John Cornyn,M,1952-02-02,Republican
7543,Ruben Kihuen,M,1980-04-25,Democrat
2187,Marco Rubio,M,1971-05-28,Republican


In [21]:
inner_joined_df.isnull().sum()

Unnamed: 0            0
name                  0
claim date            0
claim source         39
claim                 0
issue                 0
accuracy rating       0
id.bioguide        1182
id.wikipedia          0
name.first            0
name.middle        4196
name.last             0
bio.gender            0
bio.birthday         29
type                  0
party                 3
start                 0
end                   0
state              1182
full_name             0
dtype: int64

In [22]:
bday_perc_null = (18307/inner_joined_df.shape[0]) * 100
print(f'{round(bday_perc_null,2)}% of inner_joined_df do not have values for `bio.birthday`.')

239.46% of inner_joined_df do not have values for `bio.birthday`.


### Save both  left_joined_df and inner_joined_df to CSV file

In [23]:
# save left_joined_df with 25k rows
left_joined_df.to_csv("politifact_data_with_bio_raw.csv")

# save inner_joined_df with 7k rows
inner_joined_df.to_csv("politifact_data_with_bio.csv")